## Демонстрация метода.

Импортируем те пакеты, которые будем использовать в работе:

In [1]:
import pandas as pd
# пандас для работы с таблицами
import nltk
# нлтк для работы с текстами (лемматизация и тд.)
import matplotlib.pyplot as plt
# матплотлиб для графиком
from sklearn.feature_extraction.text import TfidfVectorizer
# для тф-идф
from nltk.corpus import stopwords
# английские стоп-слова
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

### 1. Подготовка данных:

Для демонстрации я буду использовать этот [датасет](https://www.kaggle.com/uciml/sms-spam-collection-dataset). 

Для начала откроем его, посмотрим на структуру данных: 

In [3]:
data = pd.read_csv('spam.csv', encoding = 'ISO-8859-1')

In [4]:
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


В датасете есть лишние столбцы, которые нам не нужны -- удалим их + переименуем столбцы для удобства пользования:

In [5]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [6]:
data = data.rename(columns={'v1': 'type', 'v2': 'message'}, index=str)

Проверим, что с данными все в порядке: 

In [7]:
print(data.shape)
data.head(10)

(5572, 2)


,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


### 2. Обработка сообщений:

In [8]:
from persistent.persistent import Persistent
# кастомный класс персистент
import numpy as np
# нампай для работы с коррдинатами и тд.

Теперь используя класс Persistent обработаем наши сообщения для получения persistent diagramms (предварительно их лемматизовав):

In [9]:
data.message = data.message.str.lower()

data.message = data.message.str.replace('[^\w\s]','')
# удалим лишние пробелы

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
# загрузим токенайзеры и лемматайзер

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
#функция для передачи в пандас

data['message_l'] = data.message.apply(lemmatize_text)

Посмотрим, что получилось:

In [12]:
data.head(10)

,type,message,message_l
0,ham,go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ham,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,spam,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,u dun say so early hor u c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,nah i dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, go, to, usf, he, lif..."
5,spam,freemsg hey there darling its been 3 weeks now...,"[freemsg, hey, there, darling, it, been, 3, we..."
6,ham,even my brother is not like to speak with me t...,"[even, my, brother, is, not, like, to, speak, ..."
7,ham,as per your request melle melle oru minnaminun...,"[a, per, your, request, melle, melle, oru, min..."
8,spam,winner as a valued network customer you have b...,"[winner, a, a, valued, network, customer, you,..."
9,spam,had your mobile 11 months or more u r entitled...,"[had, your, mobile, 11, month, or, more, u, r,..."


Теперь запустим приведение сообщений к виду персистент диаграмм:

In [13]:
texts_ph = []
# массив с текстами

for element in data.message_l:
    # цикл по лемматизированным и разделенным предложениям
    
    if element:
        # если строка не пустая и содержит слова
        a = Persistent(split_sent=element, min_count=1, window=4)
        # фигачим её в персистент класс
        dgms = a.persistent()
        # получаем диаграммы
        texts_ph.append(flatten([[p.birth for p in dgms[1]], [p.death for p in dgms[1]]]))
        # делаем просто длинный список точек как признаки
    else:
        # если пустая строка добавляем пустоту
        texts_ph.append([])

Посмотрим, как это выглядит:

In [14]:
text_ph_df = pd.DataFrame(texts_ph)
text_ph_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,948,949,950,951,952,953,954,955,956,957
0,-0.002164,-0.001904,-0.001666,-0.001189,-0.001121,-0.001085,-0.001042,-0.001006,-0.000904,-0.000898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.001585,-0.001416,-0.001129,-0.001074,-0.000981,-0.000902,-0.000720,-0.000480,-0.000112,0.000048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.002987,-0.002463,-0.002047,-0.001709,-0.001539,-0.001348,-0.001334,-0.001328,-0.001078,-0.001045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.002165,-0.001904,-0.001666,-0.001426,-0.001292,-0.001278,-0.001242,-0.001009,-0.001000,-0.000741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.002720,-0.002165,-0.002008,-0.001906,-0.001329,-0.001278,-0.001169,-0.001038,-0.000927,-0.000425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.002548,-0.002464,-0.001435,-0.001343,-0.001182,-0.001141,-0.001094,-0.001041,-0.000907,-0.000902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.003027,-0.002340,-0.001689,-0.001426,-0.001065,-0.001038,-0.001010,-0.000961,-0.000927,-0.000811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.003172,-0.002161,-0.001294,-0.001159,-0.001085,-0.001053,-0.000958,-0.000936,-0.000925,-0.000900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.003165,-0.002644,-0.002050,-0.001818,-0.001714,-0.001564,-0.001359,-0.001356,-0.001309,-0.001304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.002278,-0.001905,-0.001885,-0.001661,-0.001536,-0.001536,-0.001362,-0.001280,-0.001184,-0.001183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Поскольку координаты получаются разной длины, заполним NaN-ы нулями:

In [15]:
text_ph_df = text_ph_df.fillna(0)

In [16]:
text_ph_df.to_csv('X_data.csv')

In [18]:
pd.DataFrame(y).to_csv('y_data.csv')

### 3. Обучение:

In [45]:
from sklearn.linear_model import LogisticRegression
# логистическая регрессия
from sklearn.model_selection import train_test_split
# разбиение на трейн и тест

Теперь нам надо сделать X и у: 

In [17]:
X = text_ph_df
# Матрица -- датафрейм с координатами
y = pd.factorize(data.type)[0]
# заменим слова бинарными символами

Возьмем самый простой классификатор -- логистическую регрессию: 

In [44]:
lrc = LogisticRegression(verbose=True)
# аргумент verbose -- чтобы смотреть на процесс обучения

Очень топорно разобъем признаки на трейн и тест датасеты в отношении 70/30: 

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# разбили

Проверим, как разбили:

In [48]:
X_train.shape

(3900, 1300)

Все ок.

Зафитим модель: 

In [50]:
lrc.fit(X_train, y_train)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=True, warm_start=False)

Сравним результаты:

* На теренеровочной выборке:

In [55]:
lrc.score(X_train, y_train)

0.86897435897435893

* На тестовой выборке:

In [56]:
lrc.score(X_test, y_test)

0.85885167464114831

Результаты на тренеровчной и тестовой выборке различаются не сильно, значит оверфита нет. Никаких фокусов и никакой магии, все работает.